In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
btc_df = pd.read_csv('data/BL D.csv')

In [3]:
btc_df['Date'] = [datetime.strptime(ts, '%Y-%m-%dT%H:%M:%SZ') for ts in btc_df['time']]
del btc_df['time']

In [4]:
chg = btc_df.set_index('Date').diff()
chg = chg.shift(periods=0)

In [5]:
bdc = btc_df.merge(chg, on='Date')

In [6]:
bdc['pct_chg'] = 100*bdc['close_y']/bdc['close_x']

In [7]:
bdc = bdc[['Date', 'open_x', 'close_x', 'close_y', 'pct_chg']]
bdc.columns = ['Date', 'Open price','Close price', 'delta', '% delta']

In [8]:
def get_profitability(
    df, ndays=[5, 10, 15, 20, 25, 30], cutoff_margin=10, filter_date="2016-01-01"
):
    """
        This function takes in a dataframe containing daily open and close prices to estimate
        ROR after specified time periods from a given cut off date
            df : A dataframe containing daily open and close prices
            ndays : An arbitrary list of number of days that specify the intervals to estimate ROR
                        default = [5, 10, 15, 20, 25, 30]
            cutoff_margin : A candle in percentage points to identify a signal
                                default = 10
            filter_date :  All datapoints before this date are excluded

        Returns: Dataframe containing profitability estimates for every day we identify as a signal
        """
    df = df[df["Date"] > filter_date]
    df[">" + str(cutoff_margin) + "%"] = [
        True if abs(x) >= cutoff_margin else False for x in df["% delta"]
    ]
    pdf = df[df[">" + str(cutoff_margin) + "%"]]
    df.index = df["Date"]
    for n in ndays:
        pdf["%_" + str(n) + "_days"] = [(df.loc[date + timedelta(days=1)][1] - df.loc[date + timedelta(days=n)][2])
                                        /df.loc[date + timedelta(days=1)][1]
                                        for date in pdf["Date"]]
    return df

In [9]:
get_profitability(bdc)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Date,Open price,Close price,delta,% delta,>10%
Date,,,,,,
2016-01-02,2016-01-02,435.657439,435.880897,0.023482,0.005387,False
2016-01-03,2016-01-03,435.063788,430.797191,-5.083706,-1.180069,False
2016-01-04,2016-01-04,430.791875,433.752056,2.954865,0.681234,False
2016-01-05,2016-01-05,433.595529,433.130415,-0.621641,-0.143523,False
2016-01-06,2016-01-06,432.701216,429.864359,-3.266056,-0.759788,False
2016-01-07,2016-01-07,429.950524,459.628745,29.764386,6.475745,False
2016-01-08,2016-01-08,459.473654,455.530960,-4.097785,-0.899562,False
2016-01-09,2016-01-09,454.650780,448.621141,-6.909819,-1.540235,False
2016-01-10,2016-01-10,449.024645,447.824002,-0.797139,-0.178003,False
